In [1]:
import os
import random

import tensorflow as tf
from tensorflow.keras import models, layers, backend, callbacks, metrics, utils
from tensorflow import data

import cv2 as cv
import numpy as np

# Rename

In [20]:
for i in range(1, 1301):
    os.rename(fr"notebooks\Robot - test\robot\right\  ({i}).jpg", fr"notebooks\Robot - test\robot\right\{i}.jpg")

# Open CV Flow - Create flow images

In [2]:
path = fr"notebooks\robot"
pathSave = fr"notebooks\robot_cv_flow"
numberImages = len(os.listdir(fr"{path}\left"))

i = 1; error=0
frame = cv.imread(fr"{path}\left\{i}.jpg")
prvsL = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

frame = cv.imread(fr"{path}\right\{i}.jpg")
prvsR = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame)
hsv[...,1] = 255
i+=1

while i <= numberImages:
    #Left
    frame = cv.imread(fr"{path}\left\{i}.jpg")
    nextL = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

    flow = cv.calcOpticalFlowFarneback(prvsL,nextL, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    rgb_flowL = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)

    rgb_flowL = cv.normalize(rgb_flowL, dst=None, alpha=0, beta=255,norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)

    #Right
    frame = cv.imread(fr"{path}\right\{i}.jpg")
    nextR = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

    flow = cv.calcOpticalFlowFarneback(prvsR,nextR, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    rgb_flowR = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)

    rgb_flowR = cv.normalize(rgb_flowR, dst=None, alpha=0, beta=255,norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)

    if(np.any(rgb_flowL != 0) and np.any(rgb_flowR != 0)):
        cv.imwrite(f"{pathSave}/left/{i-1}.jpg", rgb_flowL)
        cv.imwrite(f"{pathSave}/right/{i-1}.jpg", rgb_flowR)
        prvsL = nextL
        prvsR = nextR
        i+=1
    else:
        error+=1
        if(error==20):
            print(f"error: {i}")
            error=0
            i+=1
            frame = cv.imread(fr"{path}\left\{i}.jpg")
            prvsL = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
            frame = cv.imread(fr"{path}\right\{i}.jpg")
            prvsR = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
            i+=1

error: 173
error: 370
error: 398
error: 829
error: 974
error: 1158
error: 1252
error: 1267
error: 1296


# Create pair of images

In [3]:
path = r"notebooks\robot_cv_flow"
pathSave = r"notebooks\robot_cv_flow_pair"
numberImages = len(os.listdir(fr"{path}\left"))

id = 1
while id <= numberImages:
    # Good pair

    l = cv.imread(f"{path}/left/{id}.jpg")
    r = cv.imread(f"{path}/right/{id}.jpg")

    l = cv.resize(l, dsize=(224, 224))
    r = cv.resize(r, dsize=(224, 224))

    cv.imwrite(f"{pathSave}/1/left/{id}.jpg", l)
    cv.imwrite(f"{pathSave}/1/right/{id}.jpg", r)

    # Wrong pair
    frame = 0
    secondImage = 0
    while(frame == 0 or secondImage not in range(1,numberImages+1)):
        frame = random.randint(-10,10)
        secondImage = id+frame
    
    r = cv.imread(f"{path}/right/{secondImage}.jpg")

    cv.imwrite(f"{pathSave}/0/left/{id}.jpg", l)
    cv.imwrite(f"{pathSave}/0/right/{id}.jpg", r)
    
    id+=1

# Siamese network - test

### Model

In [2]:
img_shape = (224,224,3)
dropout = [0,0,0,0]

left_input = layers.Input(img_shape)
right_input = layers.Input(img_shape)

# Convolutional Neural Network
cnn = models.Sequential()
cnn.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=img_shape))
cnn.add(layers.MaxPooling2D((3, 3)))
cnn.add(layers.Dropout(dropout[0]))

cnn.add(layers.Conv2D(48, (3, 3), activation='relu'))
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Dropout(dropout[1]))

cnn.add(layers.Conv2D(48, (3, 3), activation='relu'))
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Dropout(dropout[2]))

cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Dropout(dropout[3]))

cnn.add(layers.GlobalAveragePooling2D())
cnn.add(layers.Dense(48, activation='relu'))

# Generate the encodings (feature vectors) for the two images
left_feats = cnn(left_input)
right_fits = cnn(right_input)

# Add a customized layer to compute the difference between the encodings
difference_layer = layers.Lambda(lambda tensors:backend.abs(tensors[0] - tensors[1]))
distance = difference_layer([left_feats, right_fits])

# Add a dense layer with a sigmoid unit to generate the similarity score
prediction = layers.Dense(1,activation='sigmoid')(distance)

# Connect the inputs with the outputs
model = models.Model(inputs=[left_input,right_input],outputs=prediction)

# Compile the model
model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy", metrics.Recall(), metrics.Precision()])

model.load_weights("weight/weights")

In [3]:
path = r"notebooks\robot_cv_flow_pair"
numberImages = len(os.listdir(fr"{path}/0/left"))

# Get data from all the directories
ds = [] #0->leftWrong, 1->rightWrong, 2->leftPair, 3->rightPair
for label in range(0,2):
    for side in ['left', 'right']:
        tmp = utils.image_dataset_from_directory(
            fr"{path}/{label}/{side}",
            color_mode='rgb',
            image_size=(224,224),
            batch_size=None,
            labels=None,
            shuffle=False
        )
        tmp = tmp.map(lambda x: x/255)
        ds.append(tmp)

wrong = data.Dataset.zip((ds[0], ds[1]))
pair = data.Dataset.zip((ds[2], ds[3]))

# Create labels
wrongLables = tf.zeros([numberImages])
wrongLables = tf.data.Dataset.from_tensor_slices(wrongLables)
wrong = tf.data.Dataset.zip((wrong, wrongLables))

pairLabels = tf.ones([numberImages])
pairLabels = tf.data.Dataset.from_tensor_slices(pairLabels)
pair = tf.data.Dataset.zip((pair, pairLabels))

ds = data.Dataset.sample_from_datasets([pair, wrong], weights=[0.5, 0.5])
ds = ds.batch(1).prefetch(32)

Found 1299 files belonging to 1 classes.
Found 1299 files belonging to 1 classes.
Found 1299 files belonging to 1 classes.
Found 1299 files belonging to 1 classes.


In [4]:
loss, accuracy, recall, precision = model.evaluate(ds)
print(((1 + 1**2) * precision * recall) / ((1**2) * precision + recall))

2598/2598 [==============================] - 44s 17ms/step - loss: 1.1570 - accuracy: 0.6186 - recall: 0.4126 - precision: 0.7016
0.5196316132546493
